In [ ]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

import tensorflow as tf
import tensorflow_addons as tfa

from functools import partial
import optuna

xgb.set_config(verbosity=0)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
oof_b6 = pd.read_csv('../input/setiinference/SETI_inference/OOFs/B6_oof.csv')
oof_b6_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/B6_oof_TTA_V2.csv')

oof_b7 = pd.read_csv('../input/setiinference/SETI_inference/OOFs/B7_oof.csv')
oof_b7_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/B7_oof_TTA_V2.csv')

oof_v2_m_21k = pd.read_csv('../input/setiinference/SETI_inference/OOFs/v2-m-21k_oof.csv')
oof_v2_m_21k_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/v2-m-21k_oof_TTA_V2.csv')

oof_v2_l_21k_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/v2-l-21k_oof_TTA_V2.csv')

oof_f0 = pd.read_csv('../input/setiinference/SETI_inference/OOFs/f0_oof.csv')
oof_f0_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/f0_oof_TTA_V2.csv')

oof_f1 = pd.read_csv('../input/setiinference/SETI_inference/OOFs/f1_oof.csv')
oof_f1_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/f1_oof_TTA_V2.csv')

oof_f2_TTA = pd.read_csv('../input/setiinference/SETI_inference/OOFs/f2_oof_TTA_V2.csv')

In [ ]:
targets = oof_b6['True']
X = pd.DataFrame({
    'B6': oof_b6_TTA['B6_oof'],
    'B7': oof_b7_TTA['B7_oof'],
    'V2-m-21k': oof_v2_m_21k_TTA['v2_m_21k_oof'],
    'V2-l-21k': oof_v2_l_21k_TTA['v2_l_21k_oof'],
    'F0': oof_f0_TTA['f0_oof'],
    'F1': oof_f1_TTA['f1_oof'],
    'F2': oof_f2_TTA['f2_oof']
}).values

In [ ]:
## LGBM

# def optimize(trial, X, y):
    
#     max_depth = trial.suggest_int('max_depth', -1, 1200)
#     pos_bagging_fraction = trial.suggest_uniform('pos_bagging_fraction', 1e-4, 1)
#     neg_bagging_fraction = trial.suggest_uniform('neg_bagging_fraction', 1e-4, 1)
#     num_leaves = trial.suggest_int('num_leaves', 2, 300)
#     subsample = trial.suggest_uniform('subsample', 0.01, 1)
#     colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.01, 1)
#     reg_lambda = trial.suggest_uniform('reg_lambda', 0.01, 100)
#     reg_alpha = trial.suggest_uniform('reg_alpha', 0.01, 100)
#     scale_pos_weight = trial.suggest_uniform('scale_pos_weight', 0.01, 50)
#     max_bin = trial.suggest_int('max_bin', 50, 800)
    
    
#     params = {
#         'metric': 'auc',
#         'objective': 'binary',
#         'boosting_type': 'gbdt',
#         'max_depth': max_depth,
#         'pos_bagging_fraction': pos_bagging_fraction,
#         'neg_bagging_fraction': neg_bagging_fraction,
#         'num_leaves': num_leaves,
#         'subsample': subsample,
#         'colsample_bytree': colsample_bytree,
#         'reg_lambda': reg_lambda,
#         'reg_alpha': reg_alpha,
#         'scale_pos_weight': scale_pos_weight,
#         'max_bin': max_bin,
#         'nthreads': -1,
#         'verbosity': 0, 
#         'random_state': 42, 
#     }
    
#     oof_preds = np.zeros(len(y))
    
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        
#         X_train, X_valid = X[train_idx], X[valid_idx]
#         y_train, y_valid = y[train_idx], y[valid_idx]
        
#         LGBMTrain = lgb.Dataset(data=X_train, label=y_train)
#         LGBMValid = lgb.Dataset(data=X_valid, label=y_valid)
    
#         lgbm = lgb.train(
#             params,
#             LGBMTrain,
#             num_boost_round=15000,
#             valid_sets=[LGBMValid],
#             early_stopping_rounds=50, 
#             verbose_eval=False    
#         )
        
#         preds = lgbm.predict(X_valid)   
#         oof_preds[valid_idx] = preds

#     auc = metrics.roc_auc_score(y, oof_preds)        
#     return auc

# optimization_func = partial(optimize, X=X, y=targets)
# study = optuna.create_study(direction='maximize', study_name='best_params', storage='sqlite:///optim_lgbm.db', load_if_exists=True)
# study.optimize(optimization_func, n_trials=100, n_jobs=-1)

In [ ]:
# ## XGBoost

# def optimize(trial, X, y):
#     max_depth = trial.suggest_int('max_depth', 5, 31)
#     eta = trial.suggest_uniform('eta', 0.0001, 0.99)
#     gamma = trial.suggest_uniform('gamma', 0.01, 100)
#     min_child_weight = trial.suggest_uniform('min_child_weight', 0.01, 70)
#     max_delta_step = trial.suggest_int('max_delta_step', 1, 70)
#     subsample = trial.suggest_uniform('subsample', 0.01, 1)
#     colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.01, 1)
#     reg_lambda = trial.suggest_uniform('reg_lambda', 0.01, 70)
#     reg_alpha = trial.suggest_uniform('reg_alpha', 0.01, 70)
#     scale_pos_weight = trial.suggest_uniform('scale_pos_weight', 0.01, 50)
    
#     params = {
#         'max_depth': max_depth,
#         'eta': eta,
#         'gamma': gamma,
#         'min_child_weight': min_child_weight,
#         'max_delta_step': max_delta_step,
#         'subsample': subsample,
#         'colsample_bytree': colsample_bytree,
#         'reg_lambda': reg_lambda,
#         'reg_alpha': reg_alpha,
#         'scale_pos_weight': scale_pos_weight,
#         'random_state': 42,
#         'tree_method': 'hist',
#         'objective': 'binary:logistic'
#     }

#     oof_preds = np.zeros(len(y))
    
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        
#         X_train, X_valid = X[train_idx], X[valid_idx]
#         y_train, y_valid = y[train_idx], y[valid_idx]
        
#         DTrain = xgb.DMatrix(data=X_train, label=y_train)
#         DValid = xgb.DMatrix(data=X_valid, label=y_valid)
    
#         xgboost = xgb.train(
#             params,
#             DTrain,
#             num_boost_round=15000,
#             evals=[(DValid, 'valid')],
#             early_stopping_rounds=50,
#             verbose_eval=False
#         )
        
#         preds = xgboost.predict(xgb.DMatrix(data=X_valid)) 
#         oof_preds[valid_idx] = preds
        
#     auc = metrics.roc_auc_score(y, oof_preds)
#     return auc

# optimization_func = partial(optimize, X=X, y=targets)
# study = optuna.create_study(direction='maximize', study_name='best_params', storage='sqlite:///optim_xgb.db', load_if_exists=True)
# study.optimize(optimization_func, n_trials=100, n_jobs=-1)

In [ ]:
# ## CatBoost

# def optimize(trial, X, y):
#     learning_rate = trial.suggest_uniform('learning_rate', 1e-5, 1)
#     depth = trial.suggest_int('depth', 1, 16)
#     l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 1e-5, 100)
#     bagging_temperature = trial.suggest_uniform('bagging_temperature', 0, 500)

    
#     params = {
#         'random_state': 42,
#         'objective': 'Logloss',
#         'learning_rate': learning_rate,
#         'depth': depth,
#         'l2_leaf_reg': l2_leaf_reg,
#         'bagging_temperature': bagging_temperature,
#     }
    
#     oof_preds = np.zeros(len(y))
    
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        
#         X_train, X_valid = X[train_idx], X[valid_idx]
#         y_train, y_valid = y[train_idx], y[valid_idx]
        
#         PoolTrain = cb.Pool(data=X_train, label=y_train)
#         PoolValid = cb.Pool(data=X_valid, label=y_valid)
    
#         catboost = cb.train(
#             params=params,
#             pool=PoolTrain,
#             num_boost_round=15000,  
#             early_stopping_rounds=50,
#             evals=(PoolValid),
#             verbose=False
#         )
        
#         preds = catboost.predict(PoolValid)    
#         oof_preds[valid_idx] = preds
        
#     auc = metrics.roc_auc_score(y, oof_preds)    
#     return auc

# optimization_func = partial(optimize, X=X, y=targets)
# study = optuna.create_study(direction='maximize', study_name='best_params', storage='sqlite:///optim_xgb.db', load_if_exists=True)
# study.optimize(optimization_func, n_trials=100, n_jobs=-1)

## L1 Ensemble

In [ ]:
# lgbm_params = {
#     'metric': 'auc',
#     'objective': 'binary',
#     'boosting_type': 'gbdt',
#     'n_jobs': -1,
#     'colsample_bytree': 0.5318084187961043,
#     'max_bin': 745,
#     'max_depth': 1060,
#     'neg_bagging_fraction': 0.39104688583896124,
#     'num_leaves': 22,
#     'pos_bagging_fraction': 0.10673777166031617,
#     'reg_alpha': 59.69605015537719,
#     'reg_lambda': 90.79245721612929,
#     'scale_pos_weight': 2.219760634737945,
#     'subsample': 0.2373529814640012,
#     'verbosity': -1, 
#     'random_state': 42, 
# }

# xgb_params = {
#     'colsample_bytree': 0.8703226373638996,
#     'eta': 0.0031089968322953296,
#     'gamma': 80.80386626422809,
#     'max_delta_step': 34,
#     'max_depth': 28,
#     'min_child_weight': 12.966089078443037,
#     'reg_alpha': 0.20027402062059024,
#     'reg_lambda': 65.46303354711088,
#     'scale_pos_weight': 9.112502677070465,
#     'subsample': 0.46926622469877854,
#     'tree_method': 'hist',
#     'objective': 'binary:logistic'
# }

# cb_params = {
#     'random_state': 42,
#     'objective': 'Logloss',
#     'bagging_temperature': 80.36504941821498,
#     'depth': 6,
#     'l2_leaf_reg': 39.43502878581847,
#     'learning_rate': 0.014860578341645154
# }

In [ ]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=5, mode='max', restore_best_weights=True)
ReduceOnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_AUC', mode='max', min_lr=1e-8)

In [ ]:
lm_models = []
ann_models = []

oof_preds_lm = np.zeros(len(targets))
oof_preds_ann = np.zeros(len(targets))

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, targets)):
    print(f'Fold: {fold}')
    
    Xtrain, Xvalid = X[train_idx], X[valid_idx]
    ytrain, yvalid = targets[train_idx], targets[valid_idx]
    
    ############## LM ##############
    
    lm = LogisticRegression(random_state=42)
    lm.fit(Xtrain, ytrain)
    
    lm_models.append(lm)
    
    y_pred = lm.predict_proba(Xvalid)[:,1]
    oof_preds_lm[valid_idx] = y_pred
    
    print('LM_AUC:', metrics.roc_auc_score(yvalid, y_pred)) 
    
print('')
print('LM_OOF_AUC:', metrics.roc_auc_score(targets, oof_preds_lm))

In [ ]:
test_b6 = pd.read_csv('../input/setiinference/SETI_inference/B6_V2.csv')
test_b7 = pd.read_csv('../input/setiinference/SETI_inference/B7_V2.csv')
test_v2_m_21k = pd.read_csv('../input/setiinference/SETI_inference/v2-m-21k_V2.csv')
test_v2_l_21k = pd.read_csv('../input/setiinference/SETI_inference/v2-l-21k_V2.csv')
test_f0 = pd.read_csv('../input/setiinference/SETI_inference/NFNet-F0_V2.csv')
test_f1 = pd.read_csv('../input/setiinference/SETI_inference/NFNet-F1_V2.csv')
test_f2 = pd.read_csv('../input/setiinference/SETI_inference/NFNet-F2_V2.csv')

Xtest = pd.DataFrame({
    'B6': test_b6['target'],
    'B7': test_b7['target'],
    'V2_m_21k': test_v2_m_21k['target'],
    'V2_l_21k': test_v2_l_21k['target'],
    'F0': test_f0['target'],
    'F1': test_f1['target'],
    'F2': test_f2['target']
}).values

In [ ]:
lm_preds = np.zeros(len(Xtest))

for i in range(len(lm_models)):
    lm_preds += lm_models[i].predict_proba(Xtest)[:,1]/len(lm_models)

In [ ]:
lm = LogisticRegression(random_state=42)
lm.fit(X, targets)

final_preds = lm.predict(Xtest)

In [ ]:
submission = pd.DataFrame({
    'id': test_b6['id'],
    'target': final_preds.reshape(-1,)
})

submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()